## import文

In [3]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import warnings
import re
import lightgbm as lgb
import catboost
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.datasets import load_boston
import optuna
from functools import partial
from sklearn.model_selection import train_test_split
#交差検証に必要なライブラリ
from sklearn.model_selection import cross_validate
# スコア計算のためのライブラリ
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import roc_curve, recall_score, confusion_matrix, accuracy_score
#時刻を計測するライブラリ
import time
from sklearn.model_selection import cross_val_score
from keras import metrics

In [ ]:
warnings.simplefilter('ignore')

## データ準備

データファイルの作成と前処理

In [ ]:
#データの読み込みと連結 
path = "./train"
files = glob.glob(path+"/*.csv")

data_list = []
for file in files:
  # 全ファイルをdata_frameに変換し、その値を配列に格納する
  # index_col=0のようにindexとして使いたい列の列番号を0始まりで指定する。
    data_list.append(pd.read_csv(file, index_col=0))

# 配列に格納したデータフレームを１つにまとめる
df = pd.concat(data_list)

# データの行数とカラム数の確認
df.shape

# 欠損値やデータ型の確認
# 0 non-nullのデータは欠損値のため、削除する
df.info()

In [ ]:
# 欠損数確認
df.isnull().sum()

In [ ]:
#欠損値のみの列を削除
nonnull_list = []
for col in df.columns:
  count = df[col].count()
  if count == 0:
    nonnull_list.append(col)
df = df.drop(nonnull_list, axis = 1)  

In [ ]:
#欠損値のないデータが40万件未満の列を削除
nonnull_list = []
for col in df.columns:
  count = df[col].count()
  if count < 400000:
    nonnull_list.append(col)
df = df.drop(nonnull_list, axis = 1) 

In [ ]:
# 曖昧な表現を修正
dis = {
    "30分?60分": 45,
    "1H?1H30": 75,
    "2H?": 120,
    "1H30?2H": 105
}
df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

In [ ]:
# 市区町村コードと地区名を合わせたtown_list列を新規に作成
df['town_list'] = df['市区町村コード'].astype(str).str.cat(df['地区名'],sep='_')

# 各欠損値を同地区の平均で補完（また不要な列などは削除しておく）,ここでは最寄り駅の例のみ示す
flg_is_null = df["最寄駅：距離（分）"].isnull()
for trg in list(df.loc[flg_is_null,"town_list"].unique()):
    station_kyori = df.loc[(~flg_is_null) & (df["town_list"] == trg),"最寄駅：距離（分）"].mean()
    df["最寄駅：距離（分）"].loc[(flg_is_null) & (df["town_list"] == trg)] = station_kyori

その他、要素が文字のものは数値に変換している（平成7年→1995など）

In [ ]:
df_train = pd.read_csv("./train_merge.csv")
df_test = pd.read_csv("./test.csv")

X_train = train_te['value'] 
Y_train = train_te.drop(["value"],axis=1)
test = df_test

## モデル構築と学習

##### XGBoost

In [ ]:
class Model_XGB:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        xgb_params = {'objective': 'reg:squarederror',
                      'max_depth':108,
                      'max_leaves':273,
                      'learning_rate':0.2,
                      'n_estimators':25000,
                      'eta':0.0006,
                      'gamma':0.18,
                      'min_child_weight':0.005,
                      'max_delta_step':0.28,
                      'subsample':0.77,
                      'reg_lambda':94.8,
                      'reg_alpha':6.3,
                      'random_state': 10,
                     }
        dtrain = xgb.DMatrix(tr_x, label = tr_y)
        dvalid = xgb.DMatrix(va_x, label = va_y)
        evals = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(xgb_params, dtrain, num_boost_round = 10000,early_stopping_rounds = 40,verbose_eval = 200, evals = evals)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

#### lightGBM

In [ ]:
class Model_lgb:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        lgb_params = {'objective': 'regression_l1',
                      'boosting_type' : 'gbdt',
                      'metrics':'rmse',
                      'num_leaves':1813,
                      'learning_rate':0.02,
                      'n_estimators':25000,
                      'seed':10,
                      'lambda_l1':3.6,
                      'lambda_l2':0.3,
                      'tree_learner':'feature',
                      'feature_fraction':0.98,
                      'bagging_fraction':0.94,
                      'bagging_freq':1,
                      'min_child_samples':58,
                      'max_depth':102,
                      'min_data_in_leaf':45,
                      'feature_pre_filter':'false',
                     }
        
        lgb_train = lgb.Dataset(tr_x, label = tr_y)
        lgb_eval = lgb.Dataset(va_x, label = va_y,reference = lgb_train)
        self.model = lgb.train(lgb_params, lgb_train, valid_sets = lgb_eval, num_boost_round = 10000,early_stopping_rounds = 100,verbose_eval = 100)

    def predict(self, x):
        pred = self.model.predict(x,num_iteration = self.model.best_iteration)
        return pred

#### catboost

In [ ]:
class Model_catboost:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        #https://catboost.ai/docs/concepts/python-reference_catboostregressor.html
 
        cat_features_index = [0, 1, 2, 3, 4,6,7,8,9,12]
        catb = catboost.CatBoostRegressor(
                                    iterations = 10000, 
                                    learning_rate = 0.3,
                                    use_best_model = True, 
                                    random_seed = 3, 
                                    random_strength = 69,
                                    bagging_temperature = 0.092,
                                    od_type ='IncToDec',
                                    od_wait = 45,
                                    l2_leaf_reg = 3,
                                    depth = 11,
                                    loss_function = "RMSE",
                                  )
 
        self.model = catb.fit(tr_x,tr_y,eval_set = (va_x,va_y),cat_features = cat_features_index,early_stopping_rounds = 40,verbose_eval = 50)
        print(self.model.score(va_x,va_y))
    def predict(self, x):
        pred = self.model.predict(x)
        return pred

#### linearsvm

In [ ]:
class Model_LinearSVR:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        #params = {"C":np.logspace(0,1,params_cnt),"epsilon":np.logspace(-1,1,params_cnt)}
        self.model = LinearSVR(max_iter = 1000,
                               random_state = 10,
                               C = 0.1, #損失の係数（正則化係数の逆数）
                               epsilon = 0.01

                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

#### カーネルsvm

In [ ]:
class Model_KernelSVR:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        #params = {"kernel":['rbf'],"C":np.logspace(0,1,params_cnt), "epsilon":np.logspace(-1,1,params_cnt)}
        self.model = SVR(kernel = 'rbf',
                         gamma = 0.4,
                         max_iter = 1000,
                         C = 100, #損失の係数（正則化係数の逆数）
                         epsilon = 0.4,
                         degree = 1

                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred
    

#### ElasticNet

In [ ]:
class Model_ElasticNet:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        '''1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2
       ref)  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
        '''

        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = ElasticNet(
            alpha = 0.05, #L1係数
            l1_ratio = 0.5,
                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

#### KNN

In [ ]:
class Model_KNN:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        #params = {"kernel":['rbf'],"C":np.logspace(0,1,params_cnt), "epsilon":np.logspace(-1,1,params_cnt)}
        self.model = KNeighborsRegressor(n_neighbors = 13,
                                         weights = 'distance',
                                         algorithm = 'kd_tree',
                                         leaf_size = 218
                                        )

        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

#### out of fold(実行関数定義)

In [ ]:
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=3, shuffle=True, random_state=10)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

### 実行

In [ ]:
#XGBoost
model_1 = Model_XGB()
pred_train_1, pred_test_1 = predict_cv(model_1, X_train, Y_train, test)

他モデルも同様に実行、また特徴量の重要度など確認

In [ ]:
# 結果保存

'''
以下では、１層目の予測値をデータフレームにまとめている。
データフレームにまとめて、２層目の特徴量として、まとめている
'''
train_x_1 = pd.DataFrame({
                          'pred_1b_lgbm': pred_train_1,
                         })
test_x_1 = pd.DataFrame({
                          'pred_1b_lgbm': pred_test_1,
                         })


これらの出力結果を新たなる特徴量として繰り返すことでスタッキング手法になる

## その他（optuna）

In [ ]:
# lightgbm
def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.2, random_state=334)
    # LightGBMを利用するのに必要なフォーマットに変換
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_eval = lgb.Dataset(test_x, test_y)
    
    param = {
        'task': 'train',                 # タスクを訓練に設定
        'boosting_type': 'gbdt',         # GBDTを指定
        'objective': trial.suggest_categorical('objective',['regression','regression_l1']),    
        'metric': 'rmse',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-6, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-6, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 4096),
        'learning_rate' : trial.suggest_uniform('learning_rate', 0.01, 1.0),
        'tree_learner' : trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"]),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 1000),
        'max_depth':trial.suggest_int('max_depth', 2, 64),
        'seed':42,
        'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 5, 1000)
    }
    
    evals_result = {}
    """
    model = lgb.train(param,
                      lgb_train,
                      valid_sets=[lgb_train, lgb_eval],# 訓練データを訓練用にセット
                      verbose_eval=-1
                     )
    """
    # LightGBMで学習+予測
    model = lgb.LGBMRegressor(**param,random_state=0)# 追加部分
    
    # kFold交差検定で決定係数を算出し、各セットの平均値を返す
    kf = KFold(n_splits = 3, shuffle = True, random_state = 0)
    scores = cross_validate(model, X = X_train, y = y_train,scoring = 'r2',cv = kf)   

    return scores['test_score'].mean()

    """
    kf = KFold(n_splits=4, shuffle=True, random_state=0)
    scores = cross_validate(model.fit, X=train_x, y=train_y,scoring='r2',cv=kf)   
    return scores['test_score'].mean()
    """
    """  
    y_pred = model.predict(test_x)
    rmse = mean_squared_error(test_y, y_pred, squared=False)
    return rmse
    """
    
#optuna実行
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print('params:', study.best_params)
print('score:', study.best_value)

In [ ]:
# xgboost
def objective(trial):
    # データを訓練用と検証用に分割する
    train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 6174)

    # 1. パラメータと値の設定
    # 最適化したいパラメータと，パラメータがとる値の範囲を指定する
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 32),
        "max_leaves": trial.suggest_int("max_leaves", 1, 2048),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 1),
        "eta" :  trial.suggest_loguniform('eta', 1e-7, 1.0),
        "gamma" : trial.suggest_loguniform('gamma', 1e-7, 1.0),
        "min_child_weight" : trial.suggest_loguniform('min_child_weight', 1e-7, 1.0),
        "max_delta_step" : trial.suggest_loguniform('max_delta_step', 1e-7, 1.0),
        "subsample" : trial.suggest_uniform('subsample', 0.0, 1.0),
        "reg_lambda" : trial.suggest_uniform('reg_lambda', 0.0, 1000.0),
        "reg_alpha" : trial.suggest_uniform('reg_alpha', 0.0, 1000.0)
    }

    # 2. モデルの訓練と評価
    # モデルの訓練
    model = xgb.XGBRegressor(**params)
    model.fit(train_x, train_y)

    # モデルの評価
    pred = model.predict(test_x)
    score = mean_squared_error(test_y, pred)

    # 3. 目的関数の値を返す
    # 今回は回帰問題のためRMSEの値を最小化することを目的とする
    return score
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 100)
print('params:', study.best_params)
print('score:', study.best_value)
